## Data Preparation
First we import our necessary libraries

In [ ]:
%%script echo skipping # Comment this line out if you want to install these packages
!pip install pygris
!pip install folium

In [20]:
# Load packages
%load_ext autoreload
%autoreload 2
import pygris
from pygris import tracts
from matplotlib import pyplot as plt
import pandas as pd
import folium
import numpy as np
import torch
from linear_regression import RidgeRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Starting up 
As a test of concept, lets utilize the pygris library to access the CT tracts information and then let's do a simple plot to ensure it's correct.

In [ ]:
# Download geometry
ct_tracts = tracts(state = "CT", cb = True, cache = True, year = 2016)

# Display geometry
fig, ax = plt.subplots()
ct_tracts.plot(ax = ax)
plt.title("Tracts Cartographic Boundaries");

## Graphing Pop-Density
Before we begin our journey into zonal statistics and eventually creating a predictive model, we first want to understand what the population density looks like in Connecticut. We have some general hypotheses that the areas around New Haven and Hartford are going to have higher amounts of population, and we also expect to see some small pockets of communities around Connecticut.

In [ ]:
# Import tracts population data
pop = pd.read_csv("../data/population.csv")

# Convert data type so join key matches
ct_tracts["Geo_TRACT"] = ct_tracts["TRACTCE"].astype(int)

# Join attributes to geometry
tracts = ct_tracts.merge(pop, how = "inner", on='Geo_TRACT')

### Once we have our tracts, we then calculate pop density and graph 

In [ ]:
# Project tracts
tracts = tracts.to_crs("EPSG:3857")

# Calculate area in KM
tracts["Area"] = tracts.area/1000**2

# Calculate population density
tracts["PopDensity"] = tracts["SE_A00001_001"]/tracts["Area"]

In [ ]:
tracts.plot("PopDensity", legend = True);

## **maybe here we add some enlightening information about our tracts dataframe, otherwise we can remove the code cell above**

## Intro to Zonal Statistics
We are going to do quite a bit here which is being abstracted away by the [`rasterio`](https://pypi.org/project/rasterio/) package. First and foremost, we are going to load in our landcover data from [CONUS](https://www.ncei.noaa.gov/products/satellite/gridded-goes-conus) which is created from satellite date of the Continental United States.

This data comes in the form of a `.tif` file which is a filetype used for storing geographic satellite data.

The goal of zonal statistics here is relatively straightforward, we are going to do some math that involves the pixels in a given geographic satellite image. Each pixel has an associated number which itself is associated with a key. Each pixel is contained in a "tract" which is a measurement of land by the US Census. We perform mathematics like finding the mean type of pixel in a given area, the max, the minimum, etc. This arithmetic is handled by the [`rasterstats`](https://pythonhosted.org/rasterstats/) package.

In [ ]:
!pip install rasterio
import rasterio

### First steps 
Here we open our path to our file, and more importantly, we set up our data to be used in zonal statistics. `.read` turns our data into a Numpy Array. Following this we are going to `.transform` our data, which means we are going to take the pixel locations of our coordinates (row col) and map them to our spatial coordinates (x, y). These coordinate values are relative to the [CRS](https://www.ncei.noaa.gov/products/satellite/gridded-goes-conus) (Coordinate Reference System) which we defined earlier as **"EPSG:2234"**

In [ ]:
#the data can be accessed from https://coastalimagery.blob.core.windows.net/ccap-landcover/CCAP_bulk_download/High_Resolution_Land_Cover/Phase_2_Expanded_Categories/Legacy_Land_Cover_pre_2024/CONUS/ct_2016_ccap_hires_landcover_20200915.zip
raster_path = '../data/ct_2016_ccap_hires_landcover_20200915.tif'
landcover = rasterio.open(raster_path)
arr = landcover.read(1)
affine = landcover.transform

### Performing Zonal statistics
It's as simple as importing rasterstats. We have handled the important data manipulation, and now it's basically plug and play! One function to note is `.to_crs` which takes in given coordinate reference system and transforms all the points in our dataframe to match that system.

In [ ]:
#!pip install rasterstats

from rasterstats import zonal_stats

# Here is where we ensure our tracts CRS is the same as our landcover CRS
zone = tracts.to_crs(landcover.crs)
stats = zonal_stats(zone, arr, affine=affine)

## BOOM 
We did it! Taking a look at our dataframe we can see what this basic zonal statistics calculated the min, max, mean, and amount of pixels in the tract

In [ ]:
df = pd.DataFrame(stats)
df

## Advanced Zonal Statistics
The `rasterstats` library is very good at getting information from rasters, and we can in fact gain more information by using `categorical = True`. This allows to see the amount of each type of pixel at a given tract.

In [ ]:
from rasterstats import zonal_stats
df_new = zonal_stats(zone, arr, affine=affine, categorical = True)

Taking a look at our dataframe, we can confirm that each column is a type of pixel and each row is a tract

In [ ]:
df_categorical = pd.DataFrame(df_new)
df_categorical

## Visualizing Zonal Stats
Now that we have information on the amount of each pixel at a given tract, we can find the most common pixel per tract by using the function `.idxmax()` which will through each row and find the column with the largest value.

In [ ]:
df_categorical['max_type'] = df_categorical.idxmax(axis=1)
combined_df = pd.concat([tracts, df_categorical], axis=1)
combined_df['max_type'] = combined_df['max_type'].astype(str)

In [ ]:
combined_df.plot("max_type", legend = True);

In [ ]:
combined_df["geometry"]

### Saving this data
These statistics took quite a while to run, and it may be beneficial to save this data as a csv to continue running statistics in the future

In [ ]:
%%script echo skipping

combined_df.to_csv('../data/combined_data.csv', index=False)

## Fitting a Regression Model

First, we import our data.

In [2]:
# Import and display data
data = pd.read_csv("../data/combined_data.csv")
data.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,18,19,20,21,22,7,6,0,23,max_type
0,9,1,11000,1400000US09001011000,9001011000,110.0,CT,4473567,3841130,"POLYGON ((768806.438288436 564631.1307186291, ...",...,136572.0,423692.0,142589.0,1378858.0,NaN,NaN,NaN,NaN,NaN,2
1,9,1,20800,1400000US09001020800,9001020800,208.0,CT,2315472,0,"POLYGON ((779750.7629708104 593331.0275135201,...",...,NaN,NaN,27939.0,NaN,NaN,NaN,NaN,NaN,NaN,11
2,9,1,21400,1400000US09001021400,9001021400,214.0,CT,1640443,0,"POLYGON ((774314.4774130288 583462.5078830764,...",...,NaN,NaN,13728.0,NaN,NaN,NaN,NaN,NaN,NaN,2
3,9,1,22200,1400000US09001022200,9001022200,222.0,CT,1442382,117063,"POLYGON ((780699.029105046 578728.3344804952, ...",...,NaN,20584.0,80161.0,99956.0,NaN,NaN,NaN,NaN,NaN,2
4,9,1,43100,1400000US09001043100,9001043100,431.0,CT,6652660,58522,"POLYGON ((801316.652144019 604980.2196045901, ...",...,NaN,NaN,9940.0,68655.0,486.0,NaN,NaN,NaN,NaN,11


Looks like there is some missing data in tracts that contain no pixels of a certain class.
Let's impute 0 for all `NaN` values.

In [3]:
# Impute 0 for missing data
print("Before imputation, there were", pd.isnull(data.iloc[:,68:-1]).sum().sum(), "NaN values.")
data[pd.isnull(data.iloc[:,68:-1])] = 0
print("After imputation, there are", pd.isnull(data.iloc[:,68:-1]).sum().sum(), "NaN values.")
data.head()

Before imputation, there were 5774 NaN values.
After imputation, there are 0 NaN values.


,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,18,19,20,21,22,7,6,0,23,max_type
0,9,1,11000,1400000US09001011000,9001011000,110.0,CT,4473567,3841130,"POLYGON ((768806.438288436 564631.1307186291, ...",...,136572.0,423692.0,142589.0,1378858.0,0.0,0.0,0.0,0.0,0.0,2
1,9,1,20800,1400000US09001020800,9001020800,208.0,CT,2315472,0,"POLYGON ((779750.7629708104 593331.0275135201,...",...,0.0,0.0,27939.0,0.0,0.0,0.0,0.0,0.0,0.0,11
2,9,1,21400,1400000US09001021400,9001021400,214.0,CT,1640443,0,"POLYGON ((774314.4774130288 583462.5078830764,...",...,0.0,0.0,13728.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,9,1,22200,1400000US09001022200,9001022200,222.0,CT,1442382,117063,"POLYGON ((780699.029105046 578728.3344804952, ...",...,0.0,20584.0,80161.0,99956.0,0.0,0.0,0.0,0.0,0.0,2
4,9,1,43100,1400000US09001043100,9001043100,431.0,CT,6652660,58522,"POLYGON ((801316.652144019 604980.2196045901, ...",...,0.0,0.0,9940.0,68655.0,486.0,0.0,0.0,0.0,0.0,11


Now that we have complete data, we can calculate the proportion of pixels belonging to each class.

In [4]:
# Calculate total number of pixels in each tract
data["sum"] = data.iloc[:,68:-1].sum(axis = 1)

# Calculate proportion of pixels belonging to each class
data.iloc[:,68:-2] = data.iloc[:,68:-2].div(data['sum'], axis=0)

# View data
data.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,19,20,21,22,7,6,0,23,max_type,sum
0,9,1,11000,1400000US09001011000,9001011000,110.0,CT,4473567,3841130,"POLYGON ((768806.438288436 564631.1307186291, ...",...,0.069327,0.023331,0.225616,0.000000,0.0,0.0,0.0,0.0,2,6111530.0
1,9,1,20800,1400000US09001020800,9001020800,208.0,CT,2315472,0,"POLYGON ((779750.7629708104 593331.0275135201,...",...,0.000000,0.012054,0.000000,0.000000,0.0,0.0,0.0,0.0,11,2317904.0
2,9,1,21400,1400000US09001021400,9001021400,214.0,CT,1640443,0,"POLYGON ((774314.4774130288 583462.5078830764,...",...,0.000000,0.008350,0.000000,0.000000,0.0,0.0,0.0,0.0,2,1644135.0
3,9,1,22200,1400000US09001022200,9001022200,222.0,CT,1442382,117063,"POLYGON ((780699.029105046 578728.3344804952, ...",...,0.013289,0.051753,0.064533,0.000000,0.0,0.0,0.0,0.0,2,1548918.0
4,9,1,43100,1400000US09001043100,9001043100,431.0,CT,6652660,58522,"POLYGON ((801316.652144019 604980.2196045901, ...",...,0.000000,0.001484,0.010249,0.000073,0.0,0.0,0.0,0.0,11,6698858.0


In [5]:
# Separate predictors and outcome
X = data.iloc[:,68:-2]
y = data["PopDensity"]

In [6]:
# Fit model
# Doing this just for the purpose of seeing what it looks like
# We can use the results from this package to verify that our implementation is working properly
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Train and test split creation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
LR = LinearRegression()

m = LR.fit(X_train, y_train)

In [7]:
# R^2 value
m.score(X, y)

0.7598038103782259

In [8]:
# Y-intercept
m.intercept_

-0.0068998956630333215

In [9]:
# Variable coefficients
m.coef_

array([ 0.00763332,  0.00656805,  0.00707405,  0.00691324,  0.00630628,
        0.00668552,  0.00689624,  0.00668414,  0.00136049,  0.00707437,
        0.00699514,  0.00488618,  0.00683098,  0.00567747,  0.007384  ,
        0.00698698,  0.00792046, -0.10987691])

In [12]:
# Create predictions (probably should do this on a test set)
preds = LR.predict(X_test)

In [13]:
preds

array([-1.81472410e-05,  3.82960454e-04,  6.85736891e-05,  2.61247543e-04,
        2.28858508e-05,  6.28477156e-05,  7.26418964e-05,  1.90007726e-04,
        7.94679213e-06,  2.14147437e-04,  3.99318579e-05,  1.59433406e-04,
        2.40177157e-04,  1.27930067e-04,  1.39106083e-04,  2.49341562e-04,
        1.72371569e-04,  5.05449203e-05,  8.32804269e-05,  1.55036716e-04,
        6.93421314e-05,  1.96069308e-04,  1.47426111e-04, -7.37178961e-05,
        4.28725851e-05,  4.02919889e-05,  1.41676312e-05,  1.56496315e-05,
        9.22896796e-05,  1.56988102e-04,  1.79985399e-05,  8.92853867e-05,
        4.67938910e-04,  9.29006036e-05,  2.53912352e-04,  5.54284147e-04,
        2.76500032e-04, -1.66155285e-06,  8.44464973e-05,  6.18934187e-05,
        2.63547741e-04,  4.11809124e-05,  4.67261948e-04,  1.76218431e-04,
        2.00042648e-04,  1.32455968e-04,  5.90180317e-05,  4.37028797e-04,
        1.34149822e-05,  2.32375677e-04,  1.89978239e-04,  3.41555604e-06,
        8.24639276e-05,  

It would be cool to make a map of the predictions and have it next to a map of the actual values.

In [21]:
# Convert to torch tensors
X_train = torch.tensor(X_train.values)
y_train = torch.tensor(y_train.values)

In [23]:
RR = RidgeRegression()
RR.fit(X_train, y_train)
RR.mse(X_train, y_train)

tensor(8.1306e-09, dtype=torch.float64)

# To do
- move `combined_data.csv` to data directory
- set raster processing cells not to run by default
- perform cross-validation or create training and test data sets
- maybe save data as a shp rather than a csv?